# DoubleDrift

#### Project description:
_Adaptation of experiment 1 (perception) of Lisa & Cavanagh, 2015, Current Biology<br>
(http://dx.doi.org/10.1016/j.cub.2015.08.021) for the AMU Neuroscience Master APP 2024 courses._

#### Hypothesis: 
_Participants mislocalize perceptively the drifting gabor but saccade to correctly to its<br>
 physical position_
 
#### Exercice for APP2024:
_Analyse data to reach an adapted version of Figure 1 of [Lisa & Cavanagh, 2015, Current Biology](http://dx.doi.org/10.1016/j.cub.2015.08.021)_

<img src="img/Lisi_Cavanagh_2015_CB_Figure1.png" width=700 alt="Figure 1">
<!-- ![Lisi_Cavanagh_2015_CB_Figure1.png]()  -->

#### Eye movement data analysis:

- [x] Step 1. Extract time series

#### Step 1: extract time series

In [1]:
# Imports
import os
import numpy as np
import glob
import pandas as pd
import itertools
import scipy.io
from sac_utils import vecvel, microsacc_merge, saccpar, isincircle
import ipdb

# figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template

In [2]:
# Define folders
base_dir = '..'
data_dir = '{}/data'.format(base_dir)
subject = 'sub-01'
sessions = ['ses-01']
subject_num = subject[4:]
fig_dir = '{}/{}/figures'.format(data_dir, subject)

In [3]:
# Define data filenames
event_filenames = []
eyetrack_filnames = []
mat_filenames = []

for session in sessions:
    event_filenames.append(sorted(glob.glob('{}/{}/{}/beh/*.tsv'.format(data_dir, subject, session))))
    eyetrack_filnames.append(sorted(glob.glob('{}/{}/{}/beh/*.edf'.format(data_dir, subject, session))))
    mat_filenames.append(sorted(glob.glob('{}/{}/{}/beh/*_matlab.mat'.format(data_dir, subject, session))))
event_filenames = list(itertools.chain(*event_filenames))
eyetrack_filnames = list(itertools.chain(*eyetrack_filnames))
mat_filenames = list(itertools.chain(*mat_filenames))
num_run = len(event_filenames)

In [4]:
# Create message and data files
for eyetrack_file in eyetrack_filnames:
    
    if not os.path.exists(eyetrack_file.replace('.edf','.msg')):
        os.system('edf2asc {} -e -y'.format(eyetrack_file))
        os.rename(eyetrack_file.replace('.edf','.asc'),eyetrack_file.replace('.edf','.msg'))

    if not os.path.exists(eyetrack_file.replace('.edf','.dat')):
        os.system('edf2asc {} -s -miss -1.0 -y'.format(eyetrack_file))
        os.rename(eyetrack_file.replace('.edf','.asc'),eyetrack_file.replace('.edf','.dat'))


EDF2ASC: EyeLink EDF file -> ASCII (text) file translator
EDF2ASC version 4.2.1.0 Linux   standalone Jun 18 2021 
(c)1995-2021 by SR Research, last modified Jun 18 2021

processing file ../data/sub-01/ses-01/beh/sub-01_ses-01_task-DoubleDriftPerception_run-01_eyetrack.edf 
loadEvents = 1
Preamble of file ../data/sub-01/ses-01/beh/sub-01_ses-01_task-DoubleDriftPerception_run-01_eyetrack.edf
| DATE: Tue Sep 10 19:27:48 2013                                              |
| TYPE: EDF_FILE BINARY EVENT SAMPLE TAGGED                                   |
| VERSION: EYELINK II 1                                                       |
| SOURCE: EYELINK CL                                                          |
| EYELINK II CL v5.12 May 12 2017                                             |
| CAMERA: Eyelink GL Version 1.2 Sensor=AI7                                   |
| SERIAL NUMBER: CLG-BAF38                                                    |
| CAMERA_CONFIG: BAF38200.SCD                 

In [5]:
# Collect MSG data
msg_outputs = ['trial_onset', 'trial_offset', 'button_press_onset', 'button_left', 'button_right', 
               'fix_break', 'motion_onset', 'motion_offset', 'fixation_onset', 'fixation_offset', 
               'response_onset', 'response_offset']
num_trials = 100  # number of trials per run
 
for msg_output in msg_outputs:
    exec("{} = np.zeros(num_trials*num_run)".format(msg_output))

t_run = 0
for eyetrack_file in eyetrack_filnames:
    
    msgfid = open(eyetrack_file.replace('.edf','.msg'))
    first_last_time, first_time, last_time = False, False, False

    while not first_last_time:
        line_read = msgfid.readline()

        if not line_read == '':
            la = line_read.split()

            if len(la) > 2:
                if la[2] == 'RECORD_START' and not first_time: 
                    first_time = True
                if la[2] == 'RECORD_STOP' and not last_time:
                    last_time = True
            if len(la) > 4:
                if la[2] == 'trial' and la[4]=='check':
                    # trial %i check fixation at %f
                    trial_onset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'trial' and la[4]=='ended':
                    # trial %i ended
                    trial_offset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'trial' and la[4]=='fixation':
                    # trial %i fixation break at %f
                    fix_break[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'motion_onset':
                    # motion_onset %i at %f
                    motion_onset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'motion_offset':
                    # motion_offset %i at %f
                    motion_offset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'fixation_onset':
                    # fixation_onset %i at %f
                    fixation_onset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'fixation_offset':
                    # fixation_offset %i at %f
                    fixation_offset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'response_onset':
                    # response_onset %i at %f
                    response_onset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'response_offset':
                    # response_offset %i at %f
                    response_offset[int(la[3]) - 1 + t_run * num_trials] = float(la[1])

            if len(la) > 5:
                if la[2] == 'trial' and la[5]=='LeftArrow':
                    # trial %i event LeftArrow
                    button_press_onset[int(la[3]) -1 + t_run * num_trials] = float(la[1])
                    button_left[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
                if la[2] == 'trial' and la[5]=='RightArrow':
                    # trial %i event RightArrow
                    button_press_onset[int(la[3]) - 1 + t_run *num_trials] = float(la[1])
                    button_right[int(la[3]) - 1 + t_run * num_trials] = float(la[1])
    
        if first_time and last_time:
            first_last_time = True
            msgfid.close();
    t_run += 1

# create events dataframe
for run_num, run in enumerate(event_filenames):
    df_run = pd.read_csv(run, sep="\t")
    if run_num  > 0 :
        df_events = pd.concat([df_events, df_run])
    else :
        df_events = df_run
        

msg_dict = {}
for msg_output in msg_outputs:
    eval("msg_dict.update({'%s':%s})"%(msg_output,msg_output))

msg_dict.update({'trial_duration': trial_offset-trial_onset})
msg_dict.update({'fix_check_duration': fixation_onset-trial_onset})
msg_dict.update({'fixation_duration': fixation_offset-fixation_onset})
msg_dict.update({'response_duration': response_offset-response_onset})
msg_dict.update({'motion_duration': motion_offset-motion_onset})
msg_dict.update({'reaction_time': button_press_onset-response_onset})

df_msg = pd.DataFrame(msg_dict)
df_all = pd.concat([df_events.reset_index(drop=True),
                    df_msg.reset_index(drop=True)], axis=1)

for eyetrack_filname in eyetrack_filnames:
    os.remove(eyetrack_filname.replace('.edf','.msg'))
    os.remove(eyetrack_filname.replace('.edf','.dat'))


In [6]:
# Save dataframe
df_all.to_csv('{}/{}/{}/beh/{}_task-DoubleDriftPerception_data.csv'.format(data_dir, subject, session, subject), na_rep='n/a')

In [7]:
df_all

,onset,duration,run_number,trial_number,task,ext_mot_pos,ext_mot_ver_dir,staircase_num,fix_off_prct,trial_type,...,fixation_onset,fixation_offset,response_onset,response_offset,trial_duration,fix_check_duration,fixation_duration,response_duration,motion_duration,reaction_time
0,378.035273,3.907878,1,1,1,2,1,1,5,2,...,349249.0,352242.0,351247.0,352243.0,3908.0,917.0,2993.0,996.0,1993.0,402.0
1,381.959977,3.507757,1,2,1,1,1,2,2,1,...,352774.0,355765.0,354771.0,355765.0,3506.0,516.0,2991.0,994.0,1991.0,386.0
2,385.476196,3.507796,1,3,1,2,1,1,4,2,...,356289.0,359281.0,358288.0,359281.0,3509.0,517.0,2992.0,993.0,1992.0,285.0
3,388.992326,3.507912,1,4,1,2,2,2,3,2,...,359806.0,362797.0,361804.0,362797.0,3509.0,518.0,2991.0,993.0,1993.0,368.0
4,392.508561,3.516399,1,5,1,1,1,1,1,2,...,363332.0,366324.0,365329.0,366325.0,3519.0,528.0,2992.0,996.0,1991.0,286.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1098.563528,3.507823,2,96,1,1,2,1,2,1,...,1069373.0,1072365.0,1071372.0,1072365.0,3509.0,517.0,2992.0,993.0,1992.0,301.0
196,1102.079555,3.507836,2,97,1,2,1,1,3,2,...,1072890.0,1075881.0,1074888.0,1075881.0,3510.0,519.0,2991.0,993.0,1991.0,476.0
197,1105.595695,3.507840,2,98,1,1,2,1,3,1,...,1076406.0,1079397.0,1078404.0,1079398.0,3509.0,518.0,2991.0,994.0,1991.0,260.0
198,1109.111871,3.507837,2,99,1,1,2,1,3,1,...,1079922.0,1082913.0,1081920.0,1082913.0,3509.0,518.0,2991.0,993.0,1991.0,427.0


#### Compute psychometric function value

In [8]:
df_pf = pd.DataFrame()
angles_test = sorted(df_all.ext_mot_ori.unique())
report_right_prct = np.zeros(len(angles_test))
report_right_num = np.zeros(len(angles_test))
for angle_test_num, angle_test in enumerate(angles_test):
    report_right_prct[angle_test_num] = np.nanmean(df_all.loc[(df_all.ext_mot_ori == angle_test)]['direction_report']==2)
    report_right_num[angle_test_num] = (df_all.loc[(df_all.ext_mot_ori == angle_test)]['direction_report']==2).size

df_pf_value = pd.DataFrame()
df_pf_value['angle_test'] = angles_test
df_pf_value['angle_test'] = df_pf_value.angle_test
df_pf_value['report_right_prct'] = report_right_prct
df_pf_value['report_right_num'] = report_right_num

# Save dataframe
df_pf_value.to_csv('{}/{}/{}/beh/{}_task-DoubleDriftPerception_pfvalue.csv'.format(data_dir, subject, session, subject), na_rep='n/a')

In [9]:
df_pf_value

,angle_test,report_right_prct,report_right_num
0,10,0.000000,8.0
1,15,0.275862,29.0
2,20,0.452381,42.0
3,25,0.452381,42.0
4,30,0.609756,41.0
5,35,0.900000,20.0
6,40,0.666667,6.0
7,45,1.000000,4.0
8,50,1.000000,2.0
9,55,1.000000,2.0
